In [0]:
import numpy as np
import keras
import keras.utils
from keras import backend as K
from keras import Model
from keras import utils as np_utils
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers import Softmax, Dropout,Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/damage_dataset/train',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 256x256
        batch_size=batch_size,
        class_mode='categorical')  # 
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/damage_dataset/valid',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='categorical')


Found 1860 images belonging to 8 classes.
Found 396 images belonging to 8 classes.


In [0]:
from keras import applications
epochs = 50

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (256,256, 3))


# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(8, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001,momentum=0.9),
                    metrics=["accuracy"] )

80142336/80134624 [==============================] - 6s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("19_retrain.h5", monitor='val_acc', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [0]:
# Train the model 
history = model_final.fit_generator(
train_generator,
epochs = epochs,
steps_per_epoch = 60 ,
validation_data = validation_generator,
validation_steps = 16,
callbacks = [checkpoint, early])

Epoch 1/50


ResourceExhaustedError: ignored

# **Making Predictions**

In [0]:
import cv2
img = cv2.imread('/content/drive/My Drive/test/test4.JPEG', cv2.IMREAD_UNCHANGED)
x = cv2.resize(img, (256, 256))
y = x.reshape(-1, 256, 256, 3) 

In [0]:
from keras.models import load_model
model1 = load_model('/content/drive/My Drive/trained models/vgg19_1.h5')

In [0]:
y_prob = model1.predict(y) 
y_classes = y_prob.argmax(axis=-1) 

In [0]:
y_classes

array([0])

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_classes]
print(predictions)